In [1]:
import pandas as pd
from ivpy import attach,show,montage,histogram,scatter,compose
from ivpy.extract import extract
from ivpy.reduce import pca,tsne,umap
from ivpy.cluster import cluster
import shutil
from PIL import Image
import os
import numpy as np
import pickle
import os

from ivpy import attach,show,montage,histogram,scatter,compose
from ivpy.extract import extract
from ivpy.reduce import pca,tsne,umap
from ivpy.cluster import cluster
from ivpy.extract import norm
from sklearn.metrics import adjusted_rand_score as adjrand
from annoy import AnnoyIndex
from ivpy.data import _typecheck,_pathfilter,_featfilter,seq_types
from ivpy.plot import show

metadata = pd.read_csv("/mnt/e/Tasks/similarity/files/metadata.csv")
feature_matrix = pd.read_pickle("/mnt/e/Tasks/similarity/files/situated_views_features.pickle")
df = pd.read_pickle("/mnt/e/Tasks/similarity/files/situated_views_df.pickle")

2021-11-24 19:47:03.320642: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-24 19:47:03.320682: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
DIR = "/mnt/e/Tasks/Vistas Situadas - Imagens/classification/clusters"
df['cluster'] = cluster(feature_matrix,k=75)
for i, item in df.iterrows():   
    path = "{0}/{1}".format(DIR,item["cluster"])
    if not os.path.exists(path):
        os.makedirs(path)
    shutil.copy(item["filename"],path+"/"+item["id"]+".jpg") 

In [6]:
path_to = "/mnt/e/Tasks/Vistas Situadas - Imagens/images_cropped/duplicates_geo_ordered"

path_from = "/mnt/e/Tasks/Vistas Situadas - Imagens/images_cropped/duplicates_geo"
k_arg = -100
#affinity = 'precomputed'
def clusterization(X,df,path_to, method, karg, path_from):
    if path_from:
        folder_list = [x.split(".")[0] for x in os.listdir(path_from)]
        index_list = [df["id"].to_list().index(item.split(".")[0])for item in folder_list if item.split(".")[0] in df["id"].to_list()]
        X = X.iloc[index_list]
      
    df2 = df.copy()    
    df2["cluster"] = cluster(X,method=method, preference=karg)

    for i, item in df2.iterrows(): 
        print(item)
        path = "{0}/{1}/{2}".format(path_to,"{0}_{1}".format(method, karg),item["cluster"])    
        if not os.path.exists(path):
            os.makedirs(path)
        shutil.copy(item["filename"],path+"/"+item["id"]+".jpg") 


In [7]:

clusterization(feature_matrix,df,path_to, 'affinity', k_arg, path_from)

method: affinity
id                                             001AAN005121_geo
filename      /mnt/e/Tasks/Vistas Situadas - Imagens/images_...
geolocated                                                    0
cluster                                                     NaN
Name: 0, dtype: object
id                                             001AAN005122_geo
filename      /mnt/e/Tasks/Vistas Situadas - Imagens/images_...
geolocated                                                    0
cluster                                                     NaN
Name: 1, dtype: object
id                                             001AAN005123_geo
filename      /mnt/e/Tasks/Vistas Situadas - Imagens/images_...
geolocated                                                    0
cluster                                                     NaN
Name: 2, dtype: object
id                                             001AAN005124_geo
filename      /mnt/e/Tasks/Vistas Situadas - Imagens/images_...
geolocated        

In [ ]:
DIR = "/mnt/e/Tasks/Vistas Situadas - Imagens/images_cropped"
method='meanshift'
folder = method
folder_list = os.listdir(DIR)    
index_list = [df["id"].to_list().index(item.split(".")[0])for item in folder_list if item.split(".")[0] in df["id"].to_list()]

df['cluster_{0}'.format(folder)] = cluster(feature_matrix.iloc[index_list],method=method,  bandwidth=5)
df2 = df.dropna(subset=['cluster_{0}'.format(folder)])
for i, item in df2.iterrows(): 
    print(item)
    path_to = "{0}/{1}/{2}".format(DIR,method,item['cluster_{0}'.format(folder)])
    if item['cluster_{0}'.format(folder)] != np.nan:
        if not os.path.exists(path_to):
            os.makedirs(path_to)
        shutil.copy(item["filename"],path_to+"/"+item["id"]+".jpg") 